In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# ---------------- I/O ----------------
csv_path = Path("/mnt/data/power_bench_bf16.csv")
df = pd.read_csv(csv_path)

# --- Column detection ---
cols = {c.lower(): c for c in df.columns}

energy_mean_col = cols.get("energy_per_image_j_mean", None)
energy_std_col = cols.get("energy_per_image_j_std", None)

# Try alternate spellings for std if not exactly present
if energy_std_col is None:
    for c in df.columns:
        if c.lower().replace(" ", "_") in ("energy_per_image_j_std", "energy_per_image_j_stdev", "energy_per_image_j_sigma"):
            energy_std_col = c
            break

# Model column heuristic
model_col = None
for key in ("model", "model_name", "arch", "backbone"):
    if key in cols:
        model_col = cols[key]
        break
if model_col is None:
    object_cols = [c for c in df.columns if df[c].dtype == "object"]
    if object_cols:
        model_col = max(object_cols, key=lambda c: df[c].nunique())

# Variant column heuristic
variant_col = None
for key in ("variant", "config_variant", "run_variant"):
    if key in cols:
        variant_col = cols[key]
        break
if variant_col is None:
    # look for a column that contains 'baseline' and 'no ln' in its values
    for c in df.columns:
        vals = df[c].astype(str).str.lower().str.replace(" ", "_").str.replace("-", "_")
        if vals.isin(["baseline", "no_ln", "no_norm", "no_normalization", "without_ln"]).any():
            variant_col = c
            break

if energy_mean_col is None:
    raise RuntimeError("CSV must contain 'energy_per_image_j_mean' column. Found: " + ", ".join(df.columns))

# Normalize variant names to 'baseline' and 'no_ln'
def _norm_variant(v):
    s = str(v).strip().lower().replace(" ", "_").replace("-", "_")
    if "baseline" in s:
        return "baseline"
    if s in ("no_ln", "noln", "no_layernorm", "no_norm", "no_normalization", "no_rmsnorm", "without_ln"):
        return "no_ln"
    return s

if variant_col is None:
    df["_variant_norm"] = "baseline"
else:
    df["_variant_norm"] = df[variant_col].apply(_norm_variant)

# Keep only the two variants of interest
df = df[df["_variant_norm"].isin(["baseline", "no_ln"])].copy()

# If std isn't present, set to 0
if energy_std_col is None:
    df["_energy_std"] = 0.0
    energy_std_col = "_energy_std"

# Ensure we have model col, else synthesize
if model_col is None:
    model_col = "_model"
    df[model_col] = "model"

# --- Aggregate by (model, variant) ---
agg = (
    df.groupby([model_col, "_variant_norm"], dropna=False)
      .agg(mean=(energy_mean_col, "mean"),
           std=(energy_std_col, "mean"))  # averaging stds for display
      .reset_index()
)

# Pivot to get both variants side-by-side
pivot = agg.pivot(index=model_col, columns="_variant_norm", values=["mean", "std"])
# Drop any model missing one of the variants
pivot = pivot.dropna(subset=[("mean", "baseline"), ("mean", "no_ln")], how="any")

# Build compact summary table (as requested: baseline, no_ln, delta, delta%)
summary = pd.DataFrame({
    "model": pivot.index,
    "baseline": pivot[("mean", "baseline")].values,
    "no_ln": pivot[("mean", "no_ln")].values,
})
# stds (kept for plotting error bars)
summary["_baseline_std"] = pivot[("std", "baseline")].values
summary["_no_ln_std"] = pivot[("std", "no_ln")].values

summary["delta"] = summary["no_ln"] - summary["baseline"]
summary["delta_%"] = np.where(summary["baseline"] != 0,
                              100.0 * summary["delta"] / summary["baseline"],
                              np.nan)

# Order by baseline ascending for readability
summary = summary.sort_values("baseline").reset_index(drop=True)

# Show compact table
compact = summary[["model", "baseline", "no_ln", "delta", "delta_%"]].round(4)
# display_dataframe_to_user("Energy per image — compact summary (baseline, no_ln, delta, delta%)", compact)

# --- Slope chart ---
fig, ax = plt.subplots(figsize=(11, 6))

x_positions = {"baseline": 0.0, "no_ln": 1.0}
markers = {"baseline": "o", "no_ln": "^"}

# Use matplotlib default color cycle, but lock a consistent color per model
default_colors = plt.rcParams.get("axes.prop_cycle", None)
if default_colors is not None:
    default_colors = default_colors.by_key().get("color", None)
if not default_colors:
    default_colors = ["C0", "C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9"]

models = list(summary["model"].values)
color_map = {m: default_colors[i % len(default_colors)] for i, m in enumerate(models)}

for _, row in summary.iterrows():
    m = row["model"]
    y0 = float(row["baseline"])
    y1 = float(row["no_ln"])
    s0 = float(row["_baseline_std"])
    s1 = float(row["_no_ln_std"])
    color = color_map[m]

    # dashed line connecting baseline -> no_ln
    ax.plot(
        [x_positions["baseline"], x_positions["no_ln"]],
        [y0, y1],
        linestyle="--",
        linewidth=2,
        color=color,
        label=m,
    )

    # baseline point + error bar (marker color matches model color)
    ax.errorbar(
        x_positions["baseline"], y0, yerr=s0,
        fmt=markers["baseline"],
        capsize=4, markersize=8,
        color=color, ecolor=color, markerfacecolor=color, markeredgecolor=color
    )
    # no_ln point + error bar
    ax.errorbar(
        x_positions["no_ln"], y1, yerr=s1,
        fmt=markers["no_ln"],
        capsize=4, markersize=9,
        color=color, ecolor=color, markerfacecolor=color, markeredgecolor=color
    )

    # annotate percent change beside no_ln point
    if np.isfinite(row["delta_%"]):
        sign = "+" if row["delta_%"] >= 0 else ""
        ax.text(x_positions["no_ln"] + 0.02, y1, f"{sign}{row['delta_%']:.1f}%",
                va="center", fontsize=9)

# X ticks and labels
ax.set_xticks([x_positions["baseline"], x_positions["no_ln"]])
ax.set_xticklabels(["baseline", "no_ln"])

ax.set_ylabel("Energy per image (J)")
ax.set_title("Energy per image (mean ± std): baseline → no_ln by model")

# Legends
# Model legend (line colors)
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
leg1 = ax.legend(by_label.values(), by_label.keys(), title="Model", loc="best", frameon=True)
ax.add_artist(leg1)

# Variant legend (marker shapes only, neutral color)
variant_handles = [
    plt.Line2D([0], [0], marker=markers["baseline"], linestyle="None", label="baseline"),
    plt.Line2D([0], [0], marker=markers["no_ln"], linestyle="None", label="no_ln"),
]
ax.legend(handles=variant_handles, title="Variant", loc="upper left", frameon=True)

ax.grid(True, axis="y", linestyle="--", alpha=0.4)
ax.set_xlim(-0.1, 1.2)

plt.tight_layout()
plt.show()
# out_path = Path("/mnt/data/energy_per_image_slope.png")
# fig.savefig(out_path, dpi=200, bbox_inches="tight")
# plt.close(fig)

# out_path.__str__()


In [2]:
import sys

sys.path.append('src')

import accelerator.core
import accelerator.core.context

c:\Users\Aleksey Ryabykin\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\_internal\_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [3]:
ctx = accelerator.core.context.Context()

In [49]:
import torch
from torchvision.models import resnet18 
from accelerator.core.model import AcceleratedModel


model = resnet18()

input_example = (torch.randn(8, 3, 256, 256), )
prediction = model(*input_example)

sample = {
    'input': {
        'rgb': input_example
    },
    'target': {
        'ground_truth': torch.ones_like(prediction)
    },
}

# ctx.update_container('per_batch', sample)

wrapped  = AcceleratedModel(model)
net_result = wrapped(*sample['input'])

ctx.set_item('per_batch.prediction.net_result', net_result)

2025-10-08 18:25:09,013 - __main__.accelerator.core.model.accelerated_model - WARNING - Model collate function is set to `None`. Using default
2025-10-08 18:25:09,013 - __main__.accelerator.core.model.accelerated_model - WARNING - Model outputs separate function is set to `None`. Using default


AttributeError: 'str' object has no attribute 'shape'

In [33]:
ctx.set_model(model)

2025-10-08 15:16:02,411 - __main__.accelerator.core.model.accelerated_model - WARNING - Model collate function is set to `None`. Using default
2025-10-08 15:16:02,412 - __main__.accelerator.core.model.accelerated_model - WARNING - Model outputs separate function is set to `None`. Using default


Context(single-node, engine=None, epoch=0)
  Component Management:
    ComponentManager(1/6 active)
    Active Components:
      model: AcceleratedModel (manual)
    Available Factories:
      callbacks
      data
      distributed
      optimizer
      scheduler
    Dependencies:
      model → optimizer, scheduler
      optimizer → scheduler
  Training Management:
    TrainingManager(integrated, epoch=0, step=0)
    Component Manager Integration:
      ComponentManager(1/6 active)
      Active Components:
        model: AcceleratedModel (manual)
      Available Factories:
        callbacks
        data
        distributed
        optimizer
        scheduler
      Dependencies:
        model → optimizer, scheduler
        optimizer → scheduler
    Training Components:
      TrainingComponents(optimizer=None, scheduler=None, loss_combiner=None, distillation_manager=None, train_loop=None)
    Training State:
      TrainingState(current_epoch=0, current_step=0, global_epoch=0, global_step

In [19]:
wrapped._regularization_term

tensor([0.])

torch.Size([8, 3, 256, 256])


In [6]:
ctx.get_item('per_batch.prediction')

{'net_result': (<weakref at 0x000002B47EF75FD0; to 'Tensor' at 0x000002B45174BC40>,
  True)}

In [9]:
ctx.on_batch_end()

In [12]:
ctx.per_batch._data.update(sample)

In [13]:
ctx.set_item('per_batch.prediction.net_result', prediction)

In [11]:
ctx.per_batch._data

{'input': {}, 'prediction': {}, 'target': {}, 'additional': {}}